In [ ]:
from dotenv import load_dotenv

load_dotenv()

### STT


In [ ]:
import pyaudio

p = pyaudio.PyAudio()

info = p.get_default_host_api_info()
for i in range(info.get("deviceCount")):
    if (p.get_device_info_by_host_api_device_index(0, i).get("maxInputChannels")) > 0:
        print(
            "Input Device id ",
            i,
            " - ",
            p.get_device_info_by_host_api_device_index(0, i).get("name"),
        )

p.terminate()

In [ ]:
import wave

import pyaudio

p = pyaudio.PyAudio()

DEVICE_INDEX = 4
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 5

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    input_device_index=DEVICE_INDEX,
    frames_per_buffer=CHUNK,
)

print("recording")
wf = wave.open("./data/recording.wav", "wb")
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)

for _ in range(int(RATE / CHUNK * RECORD_SECONDS)):
    wf.writeframes(stream.read(CHUNK))

wf.close()
stream.stop_stream()
stream.close()
p.terminate()
print("done recording")

# LEFT HERE
# TODO: feed the recording into transcription model

In [ ]:
from openai import OpenAI

openai = OpenAI()

with open("./data/recording.wav", "rb") as f:
    response = openai.audio.transcriptions.create(model="whisper-1", file=f)
    print(response)

### TTS


In [ ]:
import os
import wave
from time import sleep

import pyaudio
import requests
from openai import Stream


def say(input: str | Stream):
    if isinstance(input, str):
        say_string(input)
    else:
        say_stream(input)


def say_stream(input: Stream):
    sentence_terminators = [".", "?", "!"]
    sentence = ""
    for chunk in input:
        content = chunk.choices[0].delta.content
        if content:
            for char in content:
                if (
                    char not in sentence_terminators
                    and sentence[-1:] in sentence_terminators
                ):
                    # End of sentence
                    # This blocks until playback of the sentence is done
                    # TODO: continue reading input stream while playing
                    say_string(sentence.strip())
                    sentence = char
                else:
                    sentence += char
    if sentence:
        say_string(sentence.strip())


def say_string(input: str):
    print(f"Saying: {input}")
    url = "https://api.openai.com/v1/audio/speech"
    headers = {
        "Authorization": f'Bearer {os.getenv("OPENAI_API_KEY")}',
    }

    data = {
        "model": "tts-1",
        "input": input,
        "voice": "shimmer",
        "response_format": "wav",
    }

    response = requests.post(url, headers=headers, json=data, stream=True)

    if response.ok:
        with wave.open(response.raw, "rb") as wf:

            def callback(in_data, frame_count, time_info, status):
                data = wf.readframes(frame_count)
                # If len(data) is less than requested frame_count, PyAudio automatically
                # assumes the stream is finished, and the stream stops.
                return (data, pyaudio.paContinue)

            p = pyaudio.PyAudio()
            stream = p.open(
                format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True,
                stream_callback=callback,
            )

            # Wait for stream to finish
            while stream.is_active():
                sleep(0.1)
            stream.close()
            p.terminate()
    else:
        response.raise_for_status()

In [ ]:
say("""\
I can't believe I won the lottery!
What should I do with all this money?
Should I quit my job and travel the world?
The possibilities are endless!
""")

In [ ]:
from openai import OpenAI

openai = OpenAI()

response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Write a few sentences ending with multiple ? or !"}
    ],
    stream=True,
    temperature=0,
)

say(response)